In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import datetime

In [2]:
def removeOut(dataframe):
    altered = pd.DataFrame() 
    for var in dataframe: 
        Q1 = dataframe[var].quantile(0.25) 
        Q3 = dataframe[var].quantile(0.75) 
        IQR = Q3 - Q1 
        altered[var] = dataframe[var][((dataframe[var] > (Q1 - 1.5 * IQR)) & (dataframe[var] < (Q3 + 1.5 * IQR)))] 
        altered = altered.dropna()
    return altered

In [3]:
def mean_sq_err(actual, predicted):
    return np.mean(np.square(np.array(actual) - np.array(predicted)))

In [4]:
customerDf = pd.read_csv("https://raw.githubusercontent.com/dcjh/1015_Mini_Project/main/Datasets/cleaned_customer.csv") 

In [5]:
places_df = customerDf.loc[:, ['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'Dt_Customer', 'Recency']]

In [6]:
places_df['Dt_Customer'] = pd.to_datetime(places_df['Dt_Customer'], format='%d-%m-%Y')
places_df["CurrentDate"] = datetime.datetime(2021, 1, 1)
places_df['numMonths'] = ((places_df['CurrentDate'] - places_df['Dt_Customer'])/np.timedelta64(1, 'M'))
places_df['Recency'] = places_df['Recency']/30 #assume 30days in a month

In [7]:
#average purchases over the months
for var in places_df.columns[3:9]:
    places_df[var] = places_df[var]/ (places_df["numMonths"] - places_df["Recency"])

places_df.drop(columns = ['Dt_Customer', 'CurrentDate', "numMonths", "Recency"], inplace = True)

In [8]:
places_df

,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds
0,8,10,4,6.481022,0.898157,5.572659,1.755490,0.898157,0.898157
1,1,1,2,0.136519,0.012411,0.074465,0.024822,0.012411,0.074465
2,8,2,10,4.867850,0.559917,1.451213,1.268383,0.239964,0.479929
3,2,0,4,0.134427,0.048882,0.244412,0.122206,0.036662,0.061103
4,5,3,6,2.154818,0.535591,1.469760,0.572957,0.336301,0.186834
...,...,...,...,...,...,...,...,...,...
2211,9,3,4,7.956164,0.482532,2.042344,0.471310,1.324157,2.771753
2212,8,2,5,5.280512,0.000000,0.390186,0.000000,0.000000,0.104049
2213,2,3,13,11.323378,0.598593,2.706138,0.399062,0.149648,0.299296
2214,6,5,10,5.157396,0.361500,2.578698,0.963999,0.361500,0.735049
